# Getting started with `flybody`
<a target="_blank" href="https://colab.research.google.com/github/elliottabe/mujoco_workshop2024/blob/main/Flybody_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</a>
<br>
<br>

**A Colab runtime with GPU acceleration is required.** If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".


This notebook was copied and modified from [Turaga lab github](https://github.com/TuragaLab/flybody/blob/main/docs/getting-started.ipynb)

`flybody` is an anatomically-detailed body model of the fruit fly _Drosophila melanogaster_ for MuJoCo physics simulator and reinforcement learning (RL) applications. The fly model was developed in a collaborative effort by Google DeepMind and HHMI Janelia Research Campus. We envision our model as a platform for fruit fly biophysics simulations and for modeling neural control of sensorimotor behavior in an embodied context.

This notebook presents several python examples of our MuJoCo fly model manipulations using `dm_control`.

For more background information, please explore:

* [MuJoCo documentation](https://mujoco.readthedocs.io/en/stable/overview.html)
* [MuJoCo](https://github.com/google-deepmind/mujoco) and tutorials therein
* [dm_control](https://github.com/google-deepmind/dm_control) and tutorials therein
* dm_control [paper](https://arxiv.org/abs/2006.12983)
* [PyMJCF](https://github.com/google-deepmind/dm_control/blob/main/dm_control/mjcf/README.md)

## Installing `flybody` on Colab
### _If running notebook locally, skip this cell!_

In [ ]:
# If in Colab, run this cell to install flybody. Don't forget to select GPU!!
# Otherwise, if running notebook locally, skip this cell.

import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

!pip install --upgrade pip
!python -m pip install git+https://github.com/TuragaLab/flybody.git
import sys
if 'mujoco_workshop2024' not in os.getcwd():
  !git clone https://github.com/elliottabe/mujoco_workshop2024.git
  sys.path.insert(0, os.getcwd() + '/mujoco_workshop2024')
else:
  sys.path.insert(0, os.getcwd())
# Configure MuJoCo to use the EGL rendering backend (requires GPU).
%env MUJOCO_GL=egl

# Imports

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import PIL.ImageDraw

from dm_control import mujoco
from dm_control import mjcf
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib
from tqdm import tqdm
import flybody
from flybody.fly_envs import walk_on_ball
from flybody.utils import display_video, any_substr_in_str

In [ ]:
# Frame size and camera name-to-idx mapping for rendering.
frame_size = {'width': 640, 'height': 480}
cameras = {'track1': 0, 'track2': 1, 'track3': 2,
           'back': 3, 'side': 4, 'bottom': 5, 'hero': 6,
           'eye_right': 7, 'eye_left': 8}

# Stand-alone fly model (outside of RL environment)

## Load the MuJoCo fly model
Let's load the fly model XML file `fruitfly.xml`, directly compile it to a `physics` object, and print some of the model's parameters. Note that in addition to the fly itself, the # of bodies includes one additional worldbody, joints include one freejoint connecting the fly with the worldbody, and the freejoint contributes six additional degrees of freedom (3 translation, 3 rotation).

In [ ]:
flybody_path = os.path.dirname(flybody.__file__)
xml_path = os.path.join(flybody_path, 'fruitfly/assets/fruitfly.xml')

physics = mjcf.Physics.from_xml_path(xml_path)  # Load and compile.

print('# of bodies:', physics.model.nbody)
print('# of degrees of freedom:', physics.model.nv)
print('# of joints:', physics.model.njnt)
print('# of actuators:', physics.model.nu)
print("fly's mass (gr):", physics.model.body_subtreemass[1])

## Visualize the fly as loaded
As initialized, the fly is in its default resting pose where all the joint angles stored in `physics.data.qpos` are set to zero. `physics.data.qpos` is a view onto the underlying MuJoCo's `mjData->qpos` data structure which holds the generalized coordinates of the model. Note that the entries `qpos[:3]` correspond to the Cartesian xyz position of the root freejoint in world coordinates, and `qpos[3:7]` is the quaternion representing the root joint orientation, initially set to unit quaternion `[1, 0, 0, 0]` by default. The rest of the elements `qpos[7:]` represent joint angles of all the hinge joints of our fly model.

In [ ]:
physics.data.qpos

Visualizing several views from different cameras: "hero" and "bottom" cameras

In [ ]:
pixels = physics.render(camera_id=cameras['hero'], **frame_size)
PIL.Image.fromarray(pixels)

In [ ]:
pixels = physics.render(camera_id=cameras['bottom'], **frame_size)
PIL.Image.fromarray(pixels)

We can hide the external cosmetic meshes and expose the collision geom primitives, shown in blue. Also notice the adhesive geoms shown in purple and ellipsoidal wing geoms used for fluid (air) interactions during flight simulation (orange).

In [ ]:
scene_option = mujoco.wrapper.core.MjvOption()
scene_option.geomgroup[1] = 0  # Hide external meshes.
scene_option.geomgroup[3] = 1  # Make fluid-interactions wing ellipsoids visible (orange).
scene_option.geomgroup[4] = 1  # Make collision geoms visible (blue).
pixels = physics.render(camera_id=cameras['side'], **frame_size, scene_option=scene_option)
PIL.Image.fromarray(pixels)

## Load with floor and visualize
We can also load the fly model combined with a simple flat floor (and a skybox) to make more physically meaningful simulations

In [ ]:
xml_path_floor = os.path.join(flybody_path, 'fruitfly/assets/floor.xml')
physics = mjcf.Physics.from_xml_path(xml_path_floor)

pixels = physics.render(camera_id=cameras['side'], **frame_size)
PIL.Image.fromarray(pixels)

## Kinematic manipulations
Let's try out a few simple kinematic manipulations with the fly model, i.e., without computing any forces yet, we'll put the body in several poses.

For example, we can start from the default rest position in the previous image and rotate the fly around the z-axis (vertical axis, points up) by a certain angle. This will require updating the root joint quaternion stored at `qpos[3:7]`.

For the purpose of kinematic manipulations, in addition to writing into `qpos`, we will have to update other position-related entries in MuJoCo's `mjData` data structure. One way of doing this is using `physics.reset_context()` (see [dm_control paper](https://arxiv.org/abs/2006.12983) for more detail).

In [ ]:
# Construct new root joint quaternion: 90 degree rotation around z-axis.
angle = np.pi / 2
quat = np.array([np.cos(angle/2), 0, 0, np.sin(angle/2)])
# Write into qpos and update other related quantities in mjData.
with physics.reset_context():
    physics.named.data.qpos[3:7] = quat
pixels = physics.render(camera_id=cameras['side'], **frame_size)
PIL.Image.fromarray(pixels)

Now we can try something a bit more involved.

Let's start from the default rest position again and try to fold the wings. In the model, each wing is connected to the thorax by three hinge joints (yaw, roll, pitch), effectively representing a 3-degree-of-freedom ball joint. We will need to find the wing joint angles corresponding to the folded wing position. This can be done as follows.

In addition to the actuators, most of the fly joints have weak springs attached to them. In the adsence of actuation or external forces, these springs will act to move the joints to specific pre-defined positions, such us folded wings, retracted legs for flight, holding the proboscis retracted, etc. In each case, these joint angle are stored in the joints' `springref` attribute in the XML file, and in the corresponding `qpos_spring` attribute in MuJoCo's `mjData` data structure, exposed as `physics.model.qpos_spring` through `dm_control` here.

Let's locate the wing joints and read the folded-wing joint angles from the spring parameters:

In [ ]:
wing_joints = []
folded_wing_angles = []
# Loop over all model joints.
for i in range(physics.model.njnt):
    joint_name = physics.model.id2name(i, 'joint')
    # If wing joint, store joint name and reference angle.
    if 'wing' in joint_name:
        wing_joints.append(joint_name)
        folded_wing_angles.append(
            physics.named.model.qpos_spring[joint_name].item())

wing_joints, folded_wing_angles

Now we can write these wing angles into their corresponding fields in `qpos`. Note that we are now using named (and vectorized) indexing to write the wing angles to `qpos`.

In [ ]:
with physics.reset_context():
    physics.named.data.qpos[wing_joints] = folded_wing_angles

pixels = physics.render(camera_id=cameras['side'], **frame_size)
PIL.Image.fromarray(pixels)

In the same way, we can (kinematically) put the legs into the retracted flight position, which is stored in `physics.model.qpos_spring`, as before:

In [ ]:
with physics.reset_context():
    # Loop over all joints.
    for i in range(physics.model.njnt):
        name = physics.model.id2name(i, 'joint')
        # If leg joint, set the joint angle to be equal to its corresponding
        # reference spring angle.
        if any_substr_in_str(['coxa', 'femur', 'tibia', 'tarsus'], name):
            physics.named.data.qpos[name] = physics.named.model.qpos_spring[name]
    # Also, lift the fly by setting the z-coordinate of fly's root joint.
    physics.data.qpos[2] = 1.  # Here, units: cm.

pixels = physics.render(camera_id=cameras['side'], **frame_size)
PIL.Image.fromarray(pixels)

## Kinematic replay: fold wings
Sequences of kinematically-set poses can be used for kinematic replay animations.

In this example, we will gradually move the wings from their default position to the folded position we obtained before. At each step we will render a frame and in the end generate a video from the sequence of frames:

In [ ]:
n_steps = 150
frames = []
for i in range(n_steps):
    with physics.reset_context():
        wing_angles = np.array(folded_wing_angles) * np.sin(np.pi/2 * i/n_steps)
        physics.named.data.qpos[wing_joints] = wing_angles
    pixels = physics.render(camera_id=cameras['back'], **frame_size)
    frames.append(pixels)

display_video(frames)

## Simulating physics: procedurally actuated body movements

Now we can try to run actual physical simulations.

In this example, we will procedurally control the fly's actuators and step the simulation in time to generate a sequence of movements. First, let's prepare groups of actuator names to control at each phase of the simulation. By convention, all the joint actuators in our model (in contrast to tendon actuators) have the same name as the joint they actuate.

In [ ]:
def get_leg_actuator_names(leg):
    return [f'{joint}_{leg}'
            for joint in ['coxa', 'femur', 'tibia', 'tarsus', 'tarsus2']]

# Names of leg actuators.
leg_actuators_L1 = get_leg_actuator_names('T1_left')
leg_actuators_R1 = get_leg_actuator_names('T1_right')
leg_actuators_L2 = get_leg_actuator_names('T2_left')

# Amplitudes of joint motion for front and middle legs.
amplitude_front = 0.5 * np.array([1, -1, 2, 1, 1])
amplitude_mid = 0.5 * np.array([0.5, -0.5, -2, 1, 2])

Now we can simulate the motion sequence. At each control step, we will be writing the control into MuJoCo's `mjData->ctrl`, exposed here as `physics.data.ctrl`. We will also engage the leg adhesion actuators at a certain point during the motion sequence.

The control semantics is the target joint angles for position actuators, and (scaled) force for force and adhesion actuators (see [MuJoCo docs](https://mujoco.readthedocs.io/en/stable/overview.html) for more details). With the exception of wings and adhesion, our model uses position actuators. The wings are powered by force (torque) actuators.

We will change the control once every `physics_to_ctrl_ratio == 10` simulation steps, to ensure simulation stability (see `dm_control` [paper](https://arxiv.org/abs/2006.12983) for more details).

Note that in the first part of the motion sequence, "Let wings fold", we don't alter `physics.data.ctrl` yet and it is still zero after resetting the simulation with `physics.reset()`. Nevertheless, the wings will fold -- this is achieved by the weak springs acting to move the wings to a reference position, as described above.

In [ ]:
n_steps = 100
physics_to_ctrl_ratio = 10
frames = []

# Reset physics to initial default state.
physics.reset()

# Let wings fold.
for i in range(n_steps):
    pixels = physics.render(camera_id=cameras['hero'], **frame_size)
    frames.append(pixels)
    for _ in range(physics_to_ctrl_ratio):
        physics.step()

# Twist head.
for i in range(n_steps):
    pixels = physics.render(camera_id=cameras['hero'], **frame_size)
    frames.append(pixels)
    physics.named.data.ctrl['head_twist'] = 1.5 * np.sin(2*np.pi * i/n_steps)
    for _ in range(physics_to_ctrl_ratio):
        physics.step()

# Move middle right leg.
for i in range(n_steps+50):
    if i <= n_steps:
        physics.named.data.ctrl[leg_actuators_L2] = amplitude_mid * np.sin(np.pi * i/n_steps)
    for _ in range(physics_to_ctrl_ratio):
        physics.step()
    pixels = physics.render(camera_id=cameras['hero'], **frame_size)
    frames.append(pixels)

# Activate middle leg adhision to prevent slipping when front legs are lifted later.
physics.named.data.ctrl[['adhere_claw_T2_right', 'adhere_claw_T2_left']] = 1.

# Lift fronts legs with lag.
for i in range(n_steps+100):
    left_angle = np.pi * i/n_steps
    right_angle = np.pi * i/n_steps - np.pi/5
    if left_angle <= np.pi:
        physics.named.data.ctrl[leg_actuators_L1] = amplitude_front * np.sin(left_angle)
    if 0 < right_angle <= np.pi:
        physics.named.data.ctrl[leg_actuators_R1] = amplitude_front * np.sin(right_angle)
    for _ in range(physics_to_ctrl_ratio):
        physics.step()
    pixels = physics.render(camera_id=cameras['hero'], **frame_size)
    frames.append(pixels)

display_video(frames)

## Actuate entire body with random actions
Now let's actuate all of the degrees of freedom at once with random control.

As force actuators (wings) and position actuators (the rest of the body) have different control semantics (scaled force and target joint angles, respectively), we'll actuate them with control signals of different magnitude. Let's find the indices for each actuator group first:

In [ ]:
wing_act_indices = []  # Force actuators.
body_act_indices = []  # Position actuators.
# Loop over all actuators.
for i in range(physics.model.nu):
    name = physics.model.id2name(i, 'actuator')
    # Store wing actuator indices and rest of indices separately.
    if 'wing' in name:
        wing_act_indices.append(i)
    else:
        body_act_indices.append(i)

print(wing_act_indices)
print(body_act_indices)

Run simulation and actuate the fly body with random actions.

In [ ]:
n_body_actions = len(body_act_indices)
n_wing_actions = len(wing_act_indices)

n_steps = 300
physics_to_ctrl_ratio = 10
frames = []

# Reset simulatiomn to initial default state.
physics.reset()

for i in range(n_steps):
    pixels = physics.render(camera_id=cameras['side'], **frame_size)
    frames.append(pixels)
    physics.named.data.ctrl[body_act_indices] = np.random.uniform(-.3, .3, n_body_actions)
    physics.named.data.ctrl[wing_act_indices] = np.random.uniform(-.005, .005, n_wing_actions)
    for _ in range(physics_to_ctrl_ratio):
        physics.step()

display_video(frames)

# Kinematic Replay from key points

In [ ]:
# Import custom functions from flybody and utils.
from flybody.inverse_kinematics import qpos_from_site_xpos
import Utils.io_dict_to_hdf5 as ioh5
from flybody.tasks.task_utils import retract_wings

## Model modifications: adding leg MoCap sites using [PyMJCF](https://github.com/google-deepmind/dm_control/blob/main/dm_control/mjcf/README.md)

In addition to manipulating joint angles and controls, as we did above, the fly model itself can be modified. All aspects of the model (e.g., sizes of body parts, actuator strengths and types, degrees of freedom, masses, appearance, etc.) can be easily changed programmatically.

Let's consider a simple example of fly body modifications. Assume we've got a motion capture dataset tracking the positions of the fly leg joints and we would like to fit the fly model's leg poses to this data. One way of doing so will require referencing the corresponding keypoint positions in the fly legs, which in turn can be achieved by adding sites to the leg joints.

In contrast to loading and compiling the model to a `physics` object in one step as we did before, we will split this process into two steps. First, we will load the fly model as `mjcf_model`, a python object model for the underlying MuJoCo fly XML file, which we can interact with and modify programmatically (see `dm_control`'s [PyMJCF](https://github.com/google-deepmind/dm_control/blob/main/dm_control/mjcf/README.md)). Second, we will compile the _modified_ `mjcf_model` to a `physics` object.

In [ ]:
# Site visualization parameters.
site_size = 3 * (0.005,)
site_rgba = (0, 1, 0, 1)

# Load MJCF model from the fly XML file.
mjcf_model = mjcf.from_path(xml_path)

# Make model modifications: add MoCap sites to legs. Don't compile model yet.

# Loop over bodies and add sites to position (0, 0, 0) in each body, which will
# correspond to leg joint locations.
for body in mjcf_model.find_all('body'):
    # If the body is a leg segment, add site to it.
    if any_substr_in_str(['coxa', 'femur', 'tibia', 'tarsus_', 'claw'], body.name):
        body.add('site', name=f'mocap_{body.name}', pos=(0, 0, 0),
                 size=site_size, group=0, rgba=site_rgba)

# The mjcf_model manipulations are complete, can compile now.
physics = mjcf.Physics.from_mjcf_model(mjcf_model)
physics.named.data.qpos[wing_joints] = folded_wing_angles
physics.step()
pixels = physics.render(camera_id=cameras['side'], **frame_size)
PIL.Image.fromarray(pixels)

In [ ]:
# Create names to easier access
legs = ['T1_left', 'T1_right', 'T2_left', 'T2_right', 'T3_left', 'T3_right']
joints = ['coxa', 'femur', 'tibia', 'tarsus']
xpos_geoms = ['coxa', 'femur', 'tibia', 'tarsus', 'claw']
joint_names = [f'{joint}_{leg}' for leg in legs for joint in joints]
xpos_names = [f'{joint}_{leg}' for leg in legs for joint in xpos_geoms]
site_names = [f'mocap_{joint_name}' for joint_name in xpos_names]

joints_to_manipulate = ['coxa_abduct_T1_left',  'coxa_twist_T1_left',   'coxa_T1_left',   'femur_T1_left',  'femur_twist_T1_left',  'tibia_T1_left',    'tarsus_T1_left',
                        'coxa_abduct_T1_right', 'coxa_twist_T1_right',  'coxa_T1_right',  'femur_T1_right', 'femur_twist_T1_right', 'tibia_T1_right',   'tarsus_T1_right',
                        'coxa_abduct_T2_left',  'coxa_twist_T2_left',   'coxa_T2_left',   'femur_T2_left',  'femur_twist_T2_left',  'tibia_T2_left',    'tarsus_T2_left',
                        'coxa_abduct_T2_right', 'coxa_twist_T2_right',  'coxa_T2_right',  'femur_T2_right', 'femur_twist_T2_right', 'tibia_T2_right',   'tarsus_T2_right',
                        'coxa_abduct_T3_left',  'coxa_twist_T3_left',   'coxa_T3_left',   'femur_T3_left',  'femur_twist_T3_left',  'tibia_T3_left',    'tarsus_T3_left',
                        'coxa_abduct_T3_right', 'coxa_twist_T3_right',  'coxa_T3_right',  'femur_T3_right', 'femur_twist_T3_right', 'tibia_T3_right',   'tarsus_T3_right',]

def transform_frame(frame):
    """Transform a single frame from data to model reference frame."""
    # Rotate around z-axis.
    frame = frame[:, [1, 0, 2]]
    frame[:, 1] *= -1
    # Change units mm to cm.
    frame *= 0.1
    return frame

## Load joint keypoint data

In [ ]:
if 'mujoco_workshop2024' in os.getcwd():
    datapath = './Data/kp_tracking.h5'
else:
    datapath = './mujoco_workshop2024/Data/kp_tracking.h5'
kp_tracking = ioh5.load(datapath)
# Need to align walking bout 1
kp_tracking['walking_1_kp'] = kp_tracking['walking_1_kp'] + (np.mean(kp_tracking['walking_0_kp'],axis=0) - np.mean(kp_tracking['walking_1_kp'],axis=0))
print(kp_tracking.keys())

There are two walking bouts in the dictionary:
- walking_0_kp: Shorter bout (181 timepoints)
- walking_1_kp: Longer bout (601 timepoints)

# Visualize a single frame

In [ ]:
%matplotlib inline

In [ ]:
# Set up colormap for plotting
import matplotlib as mpl
cmap = mpl.colormaps['Set1']
def map_discrete_cbar(cmap,N):
    cmap = plt.get_cmap(cmap,N+1)
    bounds = np.arange(-.5,N+1)
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
    return cmap, norm

In [ ]:
frame_idx = 0
wbout = 0
# Grab data from walking bout
test_frame = kp_tracking['walking_{}_kp'.format(wbout)][frame_idx] # (keypoint, xyz)
# Set up fly skeleton for visualization
fly_skel = np.array([(n,n+1) for n in range(29)])
fly_skel = np.stack((fly_skel[:4],fly_skel[5:9],fly_skel[10:14],fly_skel[15:19],fly_skel[20:24],fly_skel[25:30],))

cmap,norm = map_discrete_cbar('turbo',6)
fig, axs = plt.subplots(1,2,figsize=(12, 4))

ax=axs[0]
ax.scatter(test_frame[:, 0], test_frame[:, 1], c=np.arange(test_frame.shape[0]),cmap=cmap)
for n in range(fly_skel.shape[0]):
    ax.plot(test_frame[fly_skel[n], 0], test_frame[fly_skel[n], 1],c=cmap(n))
ax.axis('equal')
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_title('top-down view')

ax = axs[1]
ax.scatter(test_frame[:, 0], test_frame[:, 2], c=np.arange(test_frame.shape[0]),cmap=cmap)
for n in range(fly_skel.shape[0]):
    ax.plot(test_frame[fly_skel[n], 0], test_frame[fly_skel[n], 2],c=cmap(n))
ax.axis('equal')
ax.set_xlabel('x (mm)')
ax.set_ylabel('z (mm)')
ax.set_title('side view')
plt.show()

## Compare model's initial position with first data frame

In [ ]:
# Get the first frame.
frame0 = transform_frame(test_frame)
site_pos = physics.named.data.site_xpos[site_names]
# Shift to match origin of data and model.
frame0 += site_pos[0, :]  # Body-coxa T1 left joint is the data origin.

In [ ]:
fig, axs = plt.subplots(1,1)
ax = axs
# Plot kp positions
ax.scatter(frame0[:, 0], frame0[:, 1], c=np.arange(frame0.shape[0]), label='data',cmap=cmap)
# Plot site positions
ax.scatter(site_pos[:, 0], site_pos[:, 1], marker='x', s=100, c=np.arange(site_pos.shape[0]), label='model',cmap=cmap)
for n in range(fly_skel.shape[0]):
    ax.plot(site_pos[fly_skel[n], 0], site_pos[fly_skel[n], 1], c=cmap(n))
    ax.plot(frame0[fly_skel[n], 0], frame0[fly_skel[n], 1], c=cmap(n))
ax.axis('equal')
ax.set_title('initial site positions before fitting')
ax.set_xlabel('x (cm)')
ax.set_ylabel('y (cm)')
ax.legend()
ax.grid()

## Inverse kinematics to match keypoints and joint positions

In [ ]:
_ = physics.reset()
retract_wings(physics, prefix='')

# Run IK fit on one frame.
res = qpos_from_site_xpos(physics, site_names, frame0, joints_to_manipulate, inplace=True)

In [ ]:
# Compare site positions after fitting.
site_pos = physics.named.data.site_xpos[site_names]

fig, axs = plt.subplots(1,1)
ax = axs
ax.scatter(frame0[:, 0], frame0[:, 1], c=np.arange(frame0.shape[0]), label='data', cmap=cmap)
ax.scatter(site_pos[:, 0], site_pos[:, 1], marker='x', s=100, c=np.arange(site_pos.shape[0]), label='fitted model', cmap=cmap)
for n in range(fly_skel.shape[0]):
    ax.plot(site_pos[fly_skel[n], 0], site_pos[fly_skel[n], 1], c=cmap(n))
    ax.plot(frame0[fly_skel[n], 0], frame0[fly_skel[n], 1], c=cmap(n))
ax.axis('equal')
ax.set_title('site positions after test-fitting one frame')
ax.set_xlabel('x (cm)')
ax.set_ylabel('y (cm)')
ax.legend()
ax.grid()

# Try fitting all frames

The next cell performs inverse kinematics on all frames of a walking bout. For walking_0 it take ~4 min

In [ ]:
# Fitting all frames with inverse kinematics

# Reset physics
_ = physics.reset()
retract_wings(physics, prefix='') # retract wings
physics.step()
site_pos = physics.named.data.site_xpos[site_names] # grab initial site pose for aligning

qpos = []
all_frames = []
n_frames = len(kp_tracking['walking_{}_kp'.format(wbout)])
for i in tqdm(range(n_frames)):
    # Prepare frame for fitting.
    frame = kp_tracking['walking_{}_kp'.format(wbout)][i]  # (keypoint, xyz)
    frame = transform_frame(frame)
    frame += site_pos[0, :]  # Shift to body-coxa T1 left joint.
    all_frames.append(frame)
    # _ = physics.reset()
    # Run IK fit on current frame.
    res = qpos_from_site_xpos(physics, site_names, frame, joints_to_manipulate, inplace=True)
    # Store qpos results.
    qpos.append(res.qpos.copy())
all_frames = np.stack(all_frames)
qpos = np.stack(qpos)


In [ ]:
joint_ids = [physics.model.joint(joints_to_manipulate[n]).id+6 for n in range(len(joints_to_manipulate))] # Get joint ids from model
leg_qpos = qpos[:,joint_ids].reshape(qpos.shape[0],6,-1) # (Time,leg_num,joint_num)
fig, axs = plt.subplots(6,1,figsize=(10,15), sharex=True)
for n in range(6):
    ax = axs[n]
    ax.plot(leg_qpos[:,n,:],label=np.array(joints_to_manipulate).reshape(6,-1)[n])
    ax.legend(frameon=False,loc='upper right',bbox_to_anchor=(1.25,1),labelcolor='linecolor',handlelength=0,handleheight=0,ncols=1,columnspacing=.1)
    ax.set_ylabel('angle (rad)')
    ax.set_ylim([-np.pi,np.pi])

ax.set_xlabel('timesteps')

In [ ]:
# Generate video of fitted poses.

camera_id = 2 # Side view.

# Hide all sites.
scene_option = mujoco.wrapper.core.MjvOption()
scene_option.sitegroup[:] = [1, 1, 1, 0, 0, 0]
scene_option.flags[enums.mjtVisFlag.mjVIS_CONTACTPOINT] = True
# scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True


#retract_wings(physics)

video_frames = []
for q in tqdm(qpos):
    with physics.reset_context():
        physics.data.qpos = q.copy()
    pixels = physics.render(camera_id=cameras['hero'], width=640, height=480, scene_option=scene_option)
    video_frames.append(pixels)

In [ ]:
display_video(video_frames, framerate=30)

# Fly model in reinforcement learning environment

So far we've considered the fly model in a "bare" stand-alone configuration. One of the main applications of the model, however, is in the context of reinforcement learning (RL) environments -- training the fly to perform different tasks, such as locomotion and other behaviors.

`flybody` contains several ready-to-train RL environments, all of which can be created with a single line of code. Below we provide one simple example as a starting point.

The `composer` RL environment classes encapsulate the same fly model `physics` object we've seen before, and all our previous "bare fly model" examples and manipulations apply to RL environments just the same. As before, see [dm_control](https://github.com/google-deepmind/dm_control/) for more details on `composer` RL environments.

## Example: fly-on-ball RL environment

This RL task models a biological experimental setup where a tethered fruit fly is required to walk on a floating ball. The image below show an actual setup for such an experiment (_image credit: Igor Siwanowicz, HHMI Janelia Research Campus_).

To emulate this experimental setup, several task-specific modifications of the fly model would be required: (i) remove the root freejoint to fuse fly's thorax with the world to imitate tethering, (ii) replace the flat floor with a ball, and (iii) add an artificial observable to allow the policy to observe the rotational state of the ball. The ball position is fixed underneath the tethered fly, but it is free to rotate if a sufficient force is applied by fly's leg. The goal of the RL task is to train a policy to control the legs such that the ball rotates in a particular direction and at a particular speed.

See [task implementation](https://github.com/TuragaLab/flybody/blob/main/flybody/tasks/walk_on_ball.py) for details on the task-specific fly model modifications, reward caclulation, and RL environment step logic.

![fly-on-ball.jpg](attachment:fba018f9-7e55-4b53-a3a9-490a8b65384d.jpg)

Reinforcement learning and Deep Reinforcement learning involves how an intelligent agent ought to take actions in a dynamic environment in order to maximize the cumulative reward.

<img src="https://github.com/elliottabe/mujoco_workshop2024/blob/main/Images/RL_env.png?raw=1" alt="drawing" style="width:1000px;"/>

The task environment can be created with the following convenient one-liner:

In [ ]:
env = walk_on_ball()

We can inspect the RL environment and see the observations (policy inputs) and the actions (policy outputs). Note the extra observable `ball_qvel` we added specifically for this task. It measures the angular velocity of the ball rotation.

In [ ]:
env.observation_spec()

Action specifications: shape, data type, action names, and minima and maxima of the control ranges

In [ ]:
env.action_spec()

Let's reset the RL environment and visualize the initial state: the fly is stationary in its default pose, the wings are folded, and the ball is not rotating. This would be the initial state of a training episode or inference on a trained policy.

In [ ]:
timestep = env.reset()

pixels = env.physics.render(camera_id=cameras['track1'], **frame_size)
PIL.Image.fromarray(pixels)

## Run episode with random actions

Let's run a short episode with a dummy policy outputting random actions. As we run the environment loop we'll render frames and, for the sake of example, collect the reward at each time step.

In [ ]:
n_actions = env.action_spec().shape[0]

def random_action_policy(observation):
    del observation  # Not used by dummy policy.
    random_action = np.random.uniform(-.5, .5, n_actions)
    return random_action

frames = []
rewards = []

# Environment loop.
timestep = env.reset()
for _ in range(200):

    action = random_action_policy(timestep.observation)
    timestep = env.step(action)
    rewards.append(timestep.reward)

    pixels = env.physics.render(camera_id=cameras['track1'], **frame_size)
    frames.append(pixels)

display_video(frames)

In [ ]:
plt.plot(rewards)
plt.xlabel('timestep')
plt.ylabel('reward')